**Colab does not come with tenseal preinstalled, so please install tenseal**


*   https://github.com/OpenMined/TenSEAL



In [1]:
!pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 12.6 MB/s eta 0:00:00


**Connection to Google Drive through Colab**

In [59]:
# Run this cell if you need to connect to G drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Path to project folder**

In [83]:
%cd '/content/drive/My Drive/UNSW_CyberSecurity/UNSW_SecurityEngineering/Week 5/experiment/hm'

/content/drive/My Drive/UNSW_CyberSecurity/UNSW_SecurityEngineering/Week 5/experiment/hm


**Packages**

In [84]:
import numpy as np
import tenseal as ts
import time
import torch
import random

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm

**Model architecture**

In [85]:
class ConvNet(torch.nn.Module):
    def __init__(self, hidden=64, output=10):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3)
        self.fc1 = torch.nn.Linear(256, hidden)
        self.fc2 = torch.nn.Linear(hidden, output)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        # the model uses the square activation function
        x = self.activation(x)
        # flattening while keeping the batch axis
        x = x.view(-1, 256)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        return x

In [86]:
class EncConvNet:
    def __init__(self, torch_nn):
        self.conv1_weight = torch_nn.conv1.weight.data.view(
            torch_nn.conv1.out_channels, torch_nn.conv1.kernel_size[0],
            torch_nn.conv1.kernel_size[1]
        ).tolist()
        self.conv1_bias = torch_nn.conv1.bias.data.tolist()

        self.fc1_weight = torch_nn.fc1.weight.T.data.tolist()
        self.fc1_bias = torch_nn.fc1.bias.data.tolist()

        self.fc2_weight = torch_nn.fc2.weight.T.data.tolist()
        self.fc2_bias = torch_nn.fc2.bias.data.tolist()

    def forward(self, enc_x, windows_nb):
        # conv layer
        enc_channels = []
        for kernel, bias in zip(self.conv1_weight, self.conv1_bias):
            y = enc_x.conv2d_im2col(kernel, windows_nb) + bias
            enc_channels.append(y)
        # pack all channels into a single flattened vector
        enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        # square activation
        enc_x.polyval([0.5, 0.197, 0, -0.004])
        # fc1 layer
        enc_x = enc_x.mm(self.fc1_weight) + self.fc1_bias
        # square activation
        enc_x.polyval([0.5, 0.197, 0, -0.004])
        # fc2 layer
        enc_x = enc_x.mm(self.fc2_weight) + self.fc2_bias
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

**Load Trained Model**

In [87]:
model = ConvNet()
model.load_state_dict(torch.load(r'../model/model.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

**Load full dataset**

In [88]:
# Convert from numpy array to torch tensor.
def preprocessing(X: np.ndarray):
    X = torch.Tensor(X).type(torch.float32)
    return X.view(X.size(0), 1, 28, 28)

In [89]:
# Data class for DataLoader
class MNIST_dataset(Dataset):
    def __init__(self, input, ouput):
        self.input = input
        self.ouput = ouput

    def __getitem__(self, index):
        return self.input[index], self.ouput[index]

    def __len__ (self):
        return len(self.input)

In [90]:
ct = time.strftime('%Y%m%d-%H%M')
path = r'../data/mnist-sampled-N3500.npz'
data = np.load(path)
data_img = data.get(data.files[0]) / 255.0 # this is input data
data_lbl = data.get(data.files[1]) # this is numeric label
N = data_img.shape[0]
p = data_img.reshape(N, -1).shape[1]

data_img = preprocessing(data_img)[:30]
data_lbl = F.one_hot(torch.Tensor(data_lbl).long(), num_classes=10).float()[:30] # One-hot key encoding
criterion = nn.CrossEntropyLoss()

In [91]:
data = MNIST_dataset(data_img, data_lbl)
train_loader = DataLoader(data, batch_size=1, shuffle=True)

In [92]:
def enc_test(context, model, train_loader, criterion, kernel_shape, stride):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    for data, target in tqdm(train_loader):
        # Encoding and encryption
        x_enc, windows_nb = ts.im2col_encoding(
            context, data.view(28, 28).tolist(), kernel_shape[0],
            kernel_shape[1], stride
        )
        # Encrypted evaluation
        enc_output = enc_model(x_enc, windows_nb)
        # Decryption of result
        output = enc_output.decrypt()
        output = torch.tensor(output).view(1, -1)

        # compute loss
        loss = criterion(output, target)
        test_loss += loss.item()

        # convert output probabilities to predicted class
        _, pred = torch.max(output.data, 1)
        _, y = torch.max(target.data, 1)
        # compare predictions to true label
        correct = (pred == y).sum().item()
        # calculate accuracy for each object class
        label = y
        class_correct[label] += correct
        class_total[label] += 1


    # calculate and print avg loss
    test_loss = test_loss / sum(class_total)
    print(f'Test Loss: {test_loss:.6f}\n')

   # for label in range(10):
   #     print(
   #         f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
   #         f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
   #     )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% '
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )



# required for encoding
kernel_shape = model.conv1.kernel_size
stride = model.conv1.stride[0]

In [93]:
## Encryption Parameters

# controls precision of the fractional part
bits_scale = 26

# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)

# set the scale
context.global_scale = pow(2, bits_scale)

# galois keys are required to do ciphertext rotations
context.generate_galois_keys()

In [94]:
enc_model = EncConvNet(model)
enc_test(context, enc_model, train_loader, criterion, kernel_shape, stride)

100%|██████████| 30/30 [04:39<00:00,  9.30s/it]

Test Loss: 735.005789


Test Accuracy (Overall): 23% (7/30)


In [95]:
def enc_test2(context, model, train_loader, criterion, kernel_shape, stride):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    for data, target in tqdm(train_loader):
        output = model(data)
        # Decryption of result
        output = torch.tensor(output.detach()).view(1, -1)

        # compute loss
        loss = criterion(output, target)
        test_loss += loss.item()

        # convert output probabilities to predicted class
        _, pred = torch.max(output.data, 1)
        _, y = torch.max(target.data, 1)
        # compare predictions to true label
        correct = (pred == y).sum().item()
        # calculate accuracy for each object class
        label = y
        class_correct[label] += correct
        class_total[label] += 1


    # calculate and print avg loss
    test_loss = test_loss / sum(class_total)
    print(f'Test Loss: {test_loss:.6f}\n')

   # for label in range(10):
   #     print(
   #         f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
   #         f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
   #     )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% '
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )

In [96]:
enc_test2(context, model, train_loader, criterion, kernel_shape, stride)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-95-c9a0e36c798a>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = torch.tensor(output.detach()).view(1, -1)
100%|██████████| 30/30 [00:00<00:00, 965.18it/s]

Test Loss: 1.461445


Test Accuracy (Overall): 100% (30/30)
